In [5]:
%matplotlib widget
import ipywidgets
from IPython.display import display
import numpy as N
import matplotlib
from matplotlib import pyplot as P

# matplotlib.rc('axes', edgecolor='w')
matplotlib.rcParams['font.family'] = ['monospace']

In [ ]:
class EllipsisStack(ipywidgets.HBox):
    def __init__(self):
        super(EllipsisStack, self).__init__()
        output = ipywidgets.Output()
        self.bgcolor = 'k'
        with output:
            self.fig, self.ax = P.subplots(subplot_kw={'facecolor': self.bgcolor, 'aspect': 'equal'},
                                           constrained_layout=True, figsize=(11.693/2.0, 8.268/2.0),  # A4
                                           facecolor=self.bgcolor)
#         self.ax.set_frame_on(False)
        self.fig.canvas.toolbar_position = 'bottom'
        self.ax.tick_params(axis='x', colors=self.bgcolor, which='both')
        self.ax.tick_params(axis='y', colors=self.bgcolor, which='both')
        self.ax.yaxis.label.set_color(self.bgcolor)
        self.ax.xaxis.label.set_color(self.bgcolor)
        self.ax.set_facecolor(self.bgcolor)
        self.inserted_txt = "JACQUES PREVERT------------------------------------------------------------------------"
        self.letter_cols = ['y', 'c', 'g', 'r']

        self.n = 15
        self.a0 = 8.4
        self.b0 = 5.9
        self.np0 = 4000
        self.do_rot = True
        self.plot_text = True
        self.dt = 25.0

        # Define controls
        n_slider = ipywidgets.IntSlider(min=5, max=50, value=self.n, step=1, description="n", continuous_update=True)
        a0_slider = ipywidgets.FloatSlider(min=1.0, max=10.0, value=self.a0, step=0.1, readout_format='.1f',
                                           continuous_update=False, description="a")
        b0_slider = ipywidgets.FloatSlider(min=1.0, max=10.0, value=self.b0, step=0.1, readout_format='.1f',
                                           continuous_update=False, description="b")
        np0_slider = ipywidgets.IntSlider(min=100, max=5000, value=self.np0, step=10, description="np0",
                                          continuous_update=False)
        dt_slider = ipywidgets.FloatSlider(min=10.0, max=50.0, value=self.dt, step=1.0, readout_format='.1f',
                                           continuous_update=False, description="dt (deg)")
        rot_cb = ipywidgets.Checkbox(value=self.do_rot, description="Rotate ellipses")

        controls = ipywidgets.VBox([n_slider, a0_slider, b0_slider, np0_slider, dt_slider, rot_cb])
        controls.layout = self.make_box_layout()
        out_box = ipywidgets.Box([output])
        out_box.layout = self.make_box_layout()

        # Observe stuff
        a0_slider.observe(self.update_a0, 'value')
        b0_slider.observe(self.update_b0, 'value')
        np0_slider.observe(self.update_np0, 'value')
        n_slider.observe(self.update_n, 'value')
        dt_slider.observe(self.update_dt, 'value')
        rot_cb.observe(self.update_rot, 'value')

        # Add to children
        self.children = [controls, out_box]

        self.redraw_ellipses()

    @staticmethod
    def make_box_layout():
        return ipywidgets.Layout(border='solid 1px black', margin='0px 10px 10px 0px',
                                 padding='5px 5px 5px 5px')#, height='350px')
    def parameters(self):
        return {'n': self.n, 'a0': self.a0, 'b0': self.b0, 'np0': self.np0, 'dt': self.dt}
    
    def update_n(self, change):
        self.n = change.new
        self.redraw_ellipses()

    def update_a0(self, change):
        self.a0 = change.new
        self.redraw_ellipses()

    def update_b0(self, change):
        self.b0 = change.new
        self.redraw_ellipses()

    def update_np0(self, change):
        self.np0 = change.new
        self.redraw_ellipses()

    def update_dt(self, change):
        self.dt = change.new
        self.redraw_ellipses()

    def update_rot(self, change):
        self.do_rot = change.new
        self.redraw_ellipses()

    def redraw_ellipses(self, savefigs=False):
        for line in self.ax.lines[::-1]:
            line.remove()
        for text in self.ax.texts[::-1]:
            text.remove()
            
        self.ax.set_xlim(-1.3 * self.a0, 1.3 * self.a0)
        for i in range(self.n):
            ang = i * self.dt
            cang = N.cos(ang * N.pi / 180.0)
            sang = N.sin(ang * N.pi / 180.0)
            if i == 0:
                a = self.a0
                b = self.b0
                np = self.np0
            else:
                delta = (a / b - b / a) * N.sin(self.dt * N.pi / 180.0)
                r = N.sqrt(1 + delta ** 2 / 4.0) - delta / 2.0
                # print("r= ", r)
                a = a * r
                b = b * r
                # np = int(np*(1.0-N.sqrt(1.0-r)))
                # np = int(np * r)
                # np = int(np * (1.0 - (1.0 - r)**2))
                # np = int(np * (1.0 - (1.0 - r) ** 3))
                # print(np)

            # Random points along a 'thick' elliptical trajectory (thickness = eps)
            eps = 0.15
            p = 1.0 + N.random.uniform(size=np) * eps - eps / 2.0
            delta = N.random.uniform(size=np) * 2 * N.pi
            x = p * a * N.cos(delta)
            # y = ((p-1.0)*a + b)*N.sin(delta)
            y = p * b * N.sin(delta)
            # Rotation by angle 'ang'
            if self.do_rot:
                xrot = x * cang - y * sang
                yrot = x * sang + y * cang
            else:
                xrot = x
                yrot = y
            self.ax.plot(xrot, yrot, marker=',', color='y', linewidth=0)

            if self.plot_text:
                x = self.a0 / 4.0
                y = -self.b0 * 1.1
                if not self.do_rot:
                    xrot = x * cang + y * sang
                    yrot = -x * sang + y * cang
                else:
                    xrot = x
                    yrot = y
                    ang = 0.0
                self.ax.text(xrot, yrot, " " *i + self.inserted_txt[i], size=14, rotation=-ang,
                             ha="left", va="center", fontdict={'color': self.letter_cols[i % 4],
                                                               'family': 'Monospace'})#'Andale Mono'})

                x = -0.8 * self.a0 / N.sqrt(2.0)
                y = 1.4 * self.b0 / N.sqrt(2.0)
                if not self.do_rot:
                    xrot = x * cang + y * sang
                    yrot = -x * sang + y * cang
                else:
                    xrot = x
                    yrot = y
                    ang = 0.0
                self.ax.text(xrot, yrot, "o", size=10, rotation=-ang, ha="left", va="center",
                             fontdict={'color': 'y', 'family': 'Monospace'})
            if savefigs:
                P.subplots_adjust(0.03, 0.05, 0.98, 0.97)
                P.savefig("png/ellipsis_{iell:02d}.png".format(iell=i), facecolor='w')
                P.cla()
                
#         self.fig.canvas.draw()

    def save_figures(self):
        self.redraw_ellipses(savefigs=True)

In [ ]:
estack = EllipsisStack()
# plot = estack.children[-1]
# plot.layout.height = '600px'
display(estack)

In [32]:
estack.save_figures()

/tmp/ipykernel_73081/290301826.py:154: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  P.subplots_adjust(0.03, 0.05, 0.98, 0.97)
